# Linear Algebra Project

Ruozhen Gong

1. As demonstrated in the lecture, approximate fstar(x) = sin(x) on the interval [0, pi/2] via polynomials using a least squares approach. Use 1,000 evaluation points. Create a repository on Github holding your code. This repository needs to have a README explaining (in brief) the algorithm, as well as instructions for using your code to reproduce the results. These instructions should also include figures since a picture is worth a thousand words. Your code should be able to show results for all tasks in this project, and your code should be well-written and understandable. Use comments.

In [17]:
using WGLMakie

In [18]:
# this is the function to be approximated
npoints = 1000
x = LinRange(0, 2π, npoints)
fstar = sin.(x)

1000-element Vector{Float64}:
  0.0
  0.006289433316067751
  0.012578617838741058
  0.01886730478446709
  0.025155245389375847
  0.0314421909191206
  0.03772789267871718
  0.04401210202238166
  0.05029457036336618
  0.056575049183792345
  0.06285329004448194
  0.06912904459478454
  0.07540206458240159
  ⋮
 -0.06912904459478485
 -0.06285329004448184
 -0.056575049183792726
 -0.05029457036336704
 -0.04401210202238122
 -0.03772789267871721
 -0.031442190919121114
 -0.02515524538937595
 -0.018867304784467676
 -0.012578617838741233
 -0.006289433316067516
 -2.4492935982947064e-16

An overdetermined system is when the matrix A in Ax=b is a verticallly longer rectangle. Then there is no exact solution to the linear system. To approximately solve it, we minimize the quadratic objective function $|Ax-b|^2$ by setting the gradient to zero, the gradient of the quadratic function is a linear function, so it entails solving a linear system. 
- Least squares algorithm can solve optimization problems: min$|Ax-b|^2$ can be found by setting the gradient of that to 0, equivalently solving `A'A c = A'b`. 

In [88]:
# the least squares approach: build a matrix containing some powers of each value of x
pmax = 10
A = [x[i]^p for i in 1:npoints, p in 0:pmax-1]
# the target vector to be approximated is b, the sin function over the domain of x
b = fstar
# A is a 1000 by 10 matrix, b is a 1000-vector.

1000-element Vector{Float64}:
  0.0
  0.006289433316067751
  0.012578617838741058
  0.01886730478446709
  0.025155245389375847
  0.0314421909191206
  0.03772789267871718
  0.04401210202238166
  0.05029457036336618
  0.056575049183792345
  0.06285329004448194
  0.06912904459478454
  0.07540206458240159
  ⋮
 -0.06912904459478485
 -0.06285329004448184
 -0.056575049183792726
 -0.05029457036336704
 -0.04401210202238122
 -0.03772789267871721
 -0.031442190919121114
 -0.02515524538937595
 -0.018867304784467676
 -0.012578617838741233
 -0.006289433316067516
 -2.4492935982947064e-16

Define vector $\vec{c}=(A^T A)^{-1}(A^T b)$, the coefficient in front of each $x^p$ when $|Ax-b|^2$ is minimized. 

In [89]:
c = inv(A'*A)*(A'*b)

10-element Vector{Float64}:
  1.6206817235797644e-5
  0.9996497556567192
  0.0018161591142416
 -0.17068347707390785
  0.0047182682901620865
  0.005060368217527866
  0.0014095648657530546
 -0.0005785664834547788
  6.141031214090272e-5
 -2.1719473753718135e-6

In [90]:
f(x,c) = sum(c[p+1]*x^p for p in 0:pmax-1)

f (generic function with 1 method)

In [91]:
# Compare the approximation to the target function
fig = Figure()
Axis(fig[1,1])
xx = LinRange(0, 2π, 100)
yy = [f(x,c) for x in xx] # the approximation using least squares algorithm

# compare
scatter!(xx, sin.(xx))
lines!(xx, sin.(xx))
lines!(xx, yy)
fig

2. Define the approximation error E as the L2 norm of the difference between the approximation function f(x) and fstar(x). This is also called RMS or "root mean square" of the difference: The square root of the average of (f(x) - fstar(x))^2 at all evaluation points. What polynomial order (pmax) is required to satisfy E < 1.0e-10? Plot the
relation between pmax and E for pmax from 0 to at least 20. Discuss the figure. Can you achieve E < 1.0e-20? Say why.

In [37]:
# the approximation error for pmax = 20 is below
sum((yy.-sin.(xx)).^2)

7.271141545546218e-8

In [92]:
# now build a function to find the error that changes with pmax
function RMS(pmax) 
    npoints = 1000
    xx = LinRange(0, 2π, npoints)
    A = [xx[i]^p for i in 1:npoints, p in 0:pmax-1]
    b = sin.(xx)
    c = inv(A'*A)*(A'*b)
    yy = [sum(c[p+1]*x^p for p in 0:pmax-1) for x in xx]
    RMS = sum((yy.-sin.(xx)).^2)
    if RMS <= 1.0e-20
        print(pmax)
    end
    return RMS
end    

RMS (generic function with 1 method)

In [93]:
fig = Figure()
Axis(fig[1,1])
pmax_vals = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
RMS_vals = [RMS(p) for p in pmax_vals]
lines!(pmax_vals, RMS_vals)
fig

In [96]:
# zoom in to see the lowest RMS
fig = Figure()
Axis(fig[1,1])
pmax_vals = [7,8,9,10,11,12]
RMS_vals = [RMS(p) for p in pmax_vals]
lines!(pmax_vals, RMS_vals)
fig

In [99]:
RMS(10)

1.9006720773922263e-7

The lowest RMS did not attain 1.0e-20 for pmax from 1 to 20. The smallest error is 1.9006720773922263e-7 when we set pmax = 10. 

3. The function sin(x) is antisymmetric, i.e. sin(x) = - sin(-x). Modify the method to use only antisymmetric polynomials in your approximation. How does this affect the error? Compare results for the same computational cost, i.e. for the same number of polynomials used (not for the same pmax).

To use only antisymmetric polynomials, we use only the odd powers of x[i] to do the approximation. 

In [100]:
# modify the above function, only approximate using odd powers of the polynomial
function RMS_odd(pmax) 
    npoints = 1000
    xx = LinRange(0, 2π, npoints)
    A = [xx[i]^(2*p+1) for i in 1:npoints, p in 0:pmax-1]
    b = sin.(xx)
    c = inv(A'*A)*(A'*b)
    yy = [sum(c[p+1]*x^(2*p+1) for p in 0:pmax-1) for x in xx]
    RMS = sum((yy.-sin.(xx)).^2)
    if RMS <= 1.0e-20
        print(pmax)
    end
    return RMS
end    

RMS_odd (generic function with 1 method)

In [101]:
fig = Figure()
Axis(fig[1,1])
pmax_vals = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
RMS_odd_vals = [RMS_odd(p) for p in pmax_vals]
lines!(pmax_vals, RMS_odd_vals)
fig

In [106]:
# zoom in to see the lowest RMS value
fig = Figure()
Axis(fig[1,1])
pmax_vals = [8,9]
RMS_odd_vals = [RMS_odd(p) for p in pmax_vals]
lines!(pmax_vals, RMS_odd_vals)
fig

In [107]:
RMS_odd(9)

3.3641673981691744e-10

For the same number of polynomials used (not for the same pmax), the error smallest error is 3.3641673981691744e-10 for pmax = 9 when we use only antisymmetric polynomials in the approximation, which performs better than the previous method.

4. a. Calculate (manually, not numerically) the derivative of the approximating function $f(c, x) = \sum_p c_p x^p$. Define a new function $g(d, x) = \sum_p d_p x^p$ which depends on a different set of coefficients $|d>$. Set $g(d, x) = f'(c, x)$, and solve for the coefficients $|d>$ as a function of the coefficients $|c>$. Since the derivative is a linear operation, the relation between $|d>$ and $|c>$ can be expressed via a linear operator, the derivative matrix $D: |d> = D|c>$. Calculate D.



Derivative of $f(c, x) = \sum_{p=0}^{pmax} c_p x^p$ is $f'(c, x) =\sum_{p=0}^{pmax} p*c_p x^{(p-1)}$. Then $g(d, x) = \sum_{p=0}^{pmax} d_p x^p = \sum_{p=0}^{pmax} p*c_p x^{(p-1)}=\sum_{p=0}^{pmax-1} (p+1)*c_{p+1} x^{p}$ after rearraging the indices. 
So $d_p = (p+1)*c_{p+1}$. 

In [141]:
# we find D for the specific case where pmax = 9
pmax = 9
d = zeros(pmax)
for p in 0:(pmax-1)
    d[p+1]=(p+1)c[p+2]
end
d;

g(d,x) = sum(d[p+1]x^p for p in 0:pmax-1)

LoadError: BoundsError: attempt to access 9-element Vector{Float64} at index [10]

In [142]:
# because d has one less element than c, we have to trim c and A.
v = c[2:end]
a = A[:, 2:end]
D = zeros(pmax, pmax)
for p in 0:(pmax - 1)
    D[p+1, p+1] = p+1
end
D

9×9 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  6.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  7.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  8.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  9.0

   4. b. Calculate D numerically, and test your code by comparing (1) approximating sin(x) and calculating the derivative via D, and (2) approximating sin'(x) = cos(x). (Note that cos(x) cannot be approximated well by antisymmetric polynomials, i.e. don't use your code from task 3 above.)

In [143]:
pmax = 9
npoints = 1000
xx = LinRange(0, 2π, npoints)
A = [xx[i]^p for i in 1:npoints, p in 0:pmax-1]
b = sin.(xx)
c = inv(A'*A)*(A'*b)
yy = [sum(c[p+1]*x^p for p in 0:pmax-1) for x in xx]

1000-element Vector{Float64}:
 -0.000641370457742596
  0.005705172434811753
  0.012048992459802448
  0.018389908451050103
  0.02472773799553379
  0.031062297459727425
  0.037393402015792436
  0.04372086566762675
  0.05004450127677056
  0.056364120588169246
  0.0626795342557933
  0.06899055186811626
  0.07529698197345014
  ⋮
 -0.0689904000358283
 -0.06267937904978246
 -0.05636396196832445
 -0.05004433920236666
 -0.04372070009747402
 -0.037393232908255795
 -0.03106212477256541
 -0.02472756168607691
 -0.018389728475930612
 -0.01204880877521008
 -0.005704984996438539
  0.000641561694938901

In [144]:
# the coefficients vector d
d = D*c

9-element Vector{Float64}:
 -0.000641370457742596
  2.018561000615591
 -0.09642419870942831
 -0.4803122840821743
 -0.1730688383395318
  0.13563282674294896
 -0.022729420381438104
  0.001181222189188702
  1.831728735676617e-10

In [145]:
derivative = [sum(d[p+1]*x^p for p in 0:pmax-1) for x in xx]

1000-element Vector{Float64}:
 -0.000641370457742596
  0.012050383982722415
  0.024733789074334753
  0.03740811826891032
  0.0500726388368261
  0.06272661202467888
  0.07536929321193814
  0.08799993206659494
  0.10061777269980965
  0.1132220538195605
  0.12581200888329538
  0.13838686624958896
  0.15094584932880822
  ⋮
  6.1640636961296575
  6.181125717980347
  6.198012093260325
  6.2147231020898595
  6.231259055068304
  6.247620293569458
  6.263807190036189
  6.27982014827625
  6.295659603760993
  6.31132602392312
  6.326819908457045
  6.342141789622895

In [146]:
# zoom in to see the lowest RMS value
fig = Figure()
Axis(fig[1,1])
xx = LinRange(0, 2π, npoints)
target = cos.(xx)
lines!(xx, target)
lines!(xx, derivative)
scatter!(xx, derivative)
fig

The result is not a good approximation of cos(x).